# GPT-2

Use the HuggingFace API to fine-tune the smallest variant of GPT-2 the preprocessed Recipe Box dataset.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-18tyj7gi
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-18tyj7gi
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2221535 sha256=83523c554dd25cf9ae4bd5677a1c352c9972572bd86915b425a8040b7f2b68ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-v_fdb2rw/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers


In [3]:
!pip install -q datasets
!pip install -q transformers

In [4]:
!nvidia-smi

Tue May 11 03:18:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import pathlib
import pandas as pd

In [5]:
CACHE_DIR = "./drive/Shared drives/Capstone/tmp"
pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)
dataset_path = os.path.join(CACHE_DIR, 'recipes.pkl')

In [6]:
if not os.path.exists(dataset_path):
    raise SystemExit("Run preprocess_pickle.ipynb to generate data file before continuing")
else:
    recipes = pd.read_pickle(dataset_path)

# Subset dataset for feasible training time
# recipes = recipes[:20000]

In [9]:
recipes

,title,ingredients,instructions
0,Slow Cooker Chicken and Dumplings,"• 4 skinless, boneless chicken breast halves\n...","‣ Place the chicken, butter, soup, and onion i..."
1,Awesome Slow Cooker Pot Roast,• 2 (10.75 ounce) cans condensed cream of mush...,"‣ In a slow cooker, mix cream of mushroom soup..."
2,Brown Sugar Meatloaf,• 1/2 cup packed brown sugar\n• 1/2 cup ketchu...,‣ Preheat oven to 350 degrees F (175 degrees C...
3,Best Chocolate Chip Cookies,"• 1 cup butter, softened\n• 1 cup white sugar\...",‣ Preheat oven to 350 degrees F (175 degrees C...
4,Homemade Mac and Cheese Casserole,• 8 ounces whole wheat rotini pasta\n• 3 cups ...,‣ Preheat oven to 350 degrees F. Line a 2-quar...
...,...,...,...
125158,Cream Horns,"• 1 sheet frozen puff pastry, thawed\n• 1 egg\...",‣ Grease 8 cream horn metal cones. Cut the puf...
125159,Summer Corn Salad,• 4 ears fresh corn\n• 2 heads Belgian endive\...,‣ Watch how to make this recipe.\n‣ Preheat a ...
125160,Zucchini Stuffed Tomatoes,• 4 large plum tomatoes\n• Salt and sugar\n• 1...,‣ Preheat the broiler. Cut the tomatoes in 1/2...
125162,Chocolate Cake with Armagnac Ice Cream,• 8 ounces butter\n• 8 ounces bittersweet choc...,‣ Preheat oven to 350 degrees. On the top half...


# Fine-tuning function

https://github.com/huggingface/transformers/tree/master/examples/language-modeling

In [7]:
# Download fine-tuning script
!curl -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/language-modeling/run_clm.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20125  100 20125    0     0  55440      0 --:--:-- --:--:-- --:--:-- 55440


In [8]:
# Show all fine-tuning options
!python run_clm.py --help

2021-05-11 19:10:22.463775: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
usage: run_clm.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                  [--model_type MODEL_TYPE] [--config_name CONFIG_NAME]
                  [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                  [--no_use_fast_tokenizer]
                  [--use_fast_tokenizer [USE_FAST_TOKENIZER]]
                  [--model_revision MODEL_REVISION]
                  [--use_auth_token [USE_AUTH_TOKEN]]
                  [--dataset_name DATASET_NAME]
                  [--dataset_config_name DATASET_CONFIG_NAME]
                  [--train_file TRAIN_FILE]
                  [--validation_file VALIDATION_FILE]
                  [--max_train_samples MAX_TRAIN_SAMPLES]
                  [--max_eval_samples MAX_EVAL_SAMPLES]
                  [--block_size BLOCK_SIZE]
                  [--overwrite_cache [OVERWRITE_CACHE]]
       

In [9]:
# Hyperparameters for fine-tuning
FINETUNE_PARAMS = [
    "per_device_train_batch_size",  # Batch size per GPU/TPU core/CPU for training.
    "per_device_eval_batch_size",   # Batch size per GPU/TPU core/CPU for evaluation.
    "learning_rate",                # The initial learning rate for Adam.
    "weight_decay",                 # Weight decay if we apply some.
    "adam_epsilon",                 # Epsilon for Adam optimizer.
    "num_train_epochs",             # Total number of training epochs to perform.
    "max_steps",                    # If > 0: set total number of training steps to perform. Override num_train_epochs.
    "warmup_steps",                 # Linear warmup over warmup_steps.
    "save_steps"                    # Save checkpoint every X updates steps.
]

In [10]:
def get_model_path(model_name, nickname):
    model_filename = f"{os.path.basename(model_name)}_{nickname}"
    print(model_filename)
    return os.path.join(CACHE_DIR, model_filename)

Supports fine-tuning of any of the [hugging face causal language models](https://huggingface.co/models?filter=causal-lm) for `model_name`.

In [11]:
from sklearn.model_selection import train_test_split

def finetune(model_name, nickname, dataset, test_size=.2, **kwargs):
    # Split dataset into training and validation sets
    train_recipes, eval_recipes = train_test_split(dataset, test_size=test_size)
    print(f"{len(train_recipes)} training / {len(eval_recipes)} validation")

    # Save datasets to text files for training script
    train_file = f"train_{nickname}.txt"
    validation_file = f"eval_{nickname}.txt"

    with open(train_file, 'w') as f:
        f.write("\n".join(train_recipes))

    with open(validation_file, 'w') as f:
        f.write("\n".join(eval_recipes))

    # Get path to save model to  
    output_dir = get_model_path(model_name, nickname)

    # Parse fine-tuning parameters
    params = [f"--{arg}={value}" for (arg, value) in kwargs.items() if arg in FINETUNE_PARAMS]
    param_str = ' '.join(params)
    print(params)

    # Print out any unrecognized parameters
    unrecognized_params = [arg for arg in kwargs if arg not in FINETUNE_PARAMS]
    if len(unrecognized_params) != 0:
        print(f"Ignoring unrecognized parameters: {unrecognized_params}")
        
    # Fine-tune model
    # NOTE: Add --overwrite_output_dir to replace a model with same name
    # fp16 reduces the memory requirement of your model on the GPU by using half-precision training, basically allowing to double the batch size.
    !python run_clm.py \
        --model_name_or_path="{model_name}" \
        --train_file="{train_file}" \
        --validation_file="{validation_file}" \
        --do_train \
        --do_eval \
        --fp16 \
        --fp16_opt_level=O1 \
        --output_dir="{output_dir}" \
        {param_str}

# Text generation function

In [12]:
# Download generation script
!curl -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-generation/run_generation.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10993  100 10993    0     0  29236      0 --:--:-- --:--:-- --:--:-- 29236


In [13]:
# Show all generation options
!python run_generation.py --help

2021-05-11 19:10:40.642477: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
usage: run_generation.py [-h] --model_type MODEL_TYPE --model_name_or_path
                         MODEL_NAME_OR_PATH [--prompt PROMPT]
                         [--length LENGTH] [--stop_token STOP_TOKEN]
                         [--temperature TEMPERATURE]
                         [--repetition_penalty REPETITION_PENALTY] [--k K]
                         [--p P] [--prefix PREFIX]
                         [--padding_text PADDING_TEXT]
                         [--xlm_language XLM_LANGUAGE] [--seed SEED]
                         [--no_cuda]
                         [--num_return_sequences NUM_RETURN_SEQUENCES]
                         [--fp16]

optional arguments:
  -h, --help            show this help message and exit
  --model_type MODEL_TYPE
                        Model type selected in the list: gpt2, ctrl, openai-
                       

In [14]:
# Parameters for text generation
GENERATE_PARAMS = [
    "temperature",  # Temperature of 1.0 has no effect, lower tends toward greedy sampling
    "length",       # Length of generated text
    "prefix",       # Text added prior to input
    "stop_token"    # Token at which text generation is stopped
]

In [15]:
def generate(model_name, model_path, prompt, **kwargs):
    # Parse text generation parameters
    params = [f"--{arg}='{value}'" for (arg, value) in kwargs.items() if arg in GENERATE_PARAMS]
    param_str = ' '.join(params)
    print(params)

    # Generate text based on prompt
    !python run_generation.py \
        --model_type="{model_name}" \
        --model_name_or_path="{model_path}" \
        --prompt="{prompt}" \
        {param_str}


# Generate recipe from title

In [16]:
def recipe_to_str(recipe):
    # Combine components of recipe into a string
    return f"<BOS>{recipe.title}<ING>{recipe.ingredients}<INS>{recipe.instructions}<EOS>"

recipe_strings = recipes.apply(recipe_to_str, axis=1)

In [20]:
recipe_strings

0         <BOS>Slow Cooker Chicken and Dumplings<ING>• 4...
1         <BOS>Awesome Slow Cooker Pot Roast<ING>• 2 (10...
2         <BOS>Brown Sugar Meatloaf<ING>• 1/2 cup packed...
3         <BOS>Best Chocolate Chip Cookies<ING>• 1 cup b...
4         <BOS>Homemade Mac and Cheese Casserole<ING>• 8...
                                ...                        
125158    <BOS>Cream Horns<ING>• 1 sheet frozen puff pas...
125159    <BOS>Summer Corn Salad<ING>• 4 ears fresh corn...
125160    <BOS>Zucchini Stuffed Tomatoes<ING>• 4 large p...
125162    <BOS>Chocolate Cake with Armagnac Ice Cream<IN...
125163    <BOS>Crabby Bisque<ING>• 3 (10.5-ounce) cans r...
Length: 105791, dtype: object

## GPT-2

In [21]:
finetune('gpt2', 'fp16_recipe_105791_lr_5e-5_wd_1e-1', recipe_strings,
         per_device_train_batch_size=2, # maximum supported in GPU memory
         per_device_eval_batch_size=2,  # maximum supported in GPU memory
         learning_rate=5e-5,
         weight_decay=1e-1,
         num_train_epochs=10,
         save_steps=5000)

84632 training / 21159 validation
gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1
['--per_device_train_batch_size=2', '--per_device_eval_batch_size=2', '--learning_rate=5e-05', '--weight_decay=0.1', '--num_train_epochs=10', '--save_steps=5000']
2021-05-11 03:19:47.734055: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 03:19:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/11/2021 03:19:49 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=

## Generate recipes

In [17]:
def generate_recipe(title, ingredients=[], generate_more_ingredients=True):
    ingredients = '\n'.join(map(lambda i: f"• {i}", ingredients))
    prompt = f"{title}<ING>{ingredients}"

    if not generate_more_ingredients:
        prompt += "\n<INS>"

    model_path = get_model_path('gpt2', 'fp16_recipe_105791_lr_5e-5_wd_1e-1')
    generate('gpt2', model_path, prompt, temperature=0.7, length=400, prefix='<BOS>', stop_token='<EOS>')

In [18]:
generate_recipe("Chicken Tikka Masala")

gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 19:10:59.355714: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 19:11:00 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 19:11:06 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Chicken Tikka Masala<ING>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Chicken Tikka Masala<ING>• 1 teaspoon ground cumin• 1 teaspoon ground coriander• 1/2 teaspoon grou

In [22]:
generate_recipe("Taco")

gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 19:03:34.846929: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 19:03:36 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 19:03:42 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Taco<ING>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Taco<ING>• 1 pound ground beef• 1 (1.25 ounce) package taco seasoning mix• 1 (16 ounce) package shredded Cheddar c

In [23]:
generate_recipe("Casserole", ['pasta', 'tuna', 'cheese', 'jalapeno', 'ketchup'])

gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 19:03:57.752102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 19:03:59 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 19:04:05 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Casserole<ING>• pasta\n• tuna\n• cheese\n• jalapeno\n• ketchup', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Casserole<ING>• pasta
• tuna
• cheese
• jalapeno
• ketchup• m

In [24]:
generate_recipe("Casserole", ['pasta', 'tuna', 'cheese', 'jalapeno', 'ketchup'], False)

gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 19:04:27.660623: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 19:04:29 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 19:04:34 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Casserole<ING>• pasta\n• tuna\n• cheese\n• jalapeno\n• ketchup\n<INS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Casserole<ING>• pasta
• tuna
• cheese
• jalapeno
• ket

In [19]:
generate_recipe("Texas Pecan and Chocolate Pie", ['chocolate'])

gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 19:11:25.363677: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 19:11:26 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 19:11:32 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Texas Pecan and Chocolate Pie<ING>• chocolate', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Texas Pecan and Chocolate Pie<ING>• chocolate wafer crumbs• 1/2 cup butter• 2 

In [ ]:
generate_recipe("Mushroom Tacos", ['tortilla', 'mushroom', 'cheese', 'lettuce'], False)

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-23 01:52:30.002800: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/23/2020 01:52:32 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/23/2020 01:52:42 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Mushroom Tacos<ING>• tortilla\n• mushroom\n• cheese\n• lettuce\n<INS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Mushroom Tacos<ING>• tortilla
• mushroom
• cheese


# Generate evaluation recipes

In [20]:
# Set to True for generating recipes for evaluation
GENERATE_EVAL_RECIPES = True 

In [21]:
def generate_file(model_name, model_path, prompt, output_path, **kwargs):
    # Parse text generation parameters
    params = [f"--{arg}='{value}'" for (arg, value) in kwargs.items() if arg in GENERATE_PARAMS]
    param_str = ' '.join(params)
    print(params)

    # Generate text based on prompt
    !python run_generation.py \
        --model_type="{model_name}" \
        --model_name_or_path="{model_path}" \
        --prompt="{prompt}" \
        {param_str} \
        | tail -n +2 > "{output_path}"


In [22]:
import random
from tqdm import tqdm

if GENERATE_EVAL_RECIPES:
    recipe_titles = recipes.filter(['title']).values.ravel().tolist()
    # recipe_words = set()

    # Create vocabulary of words used in recipe titles as potential inputs for generation
    # for title in recipe_titles:
    #     for word in title.split(' '):
    #         if len(word) > 0: # Don't add empty string
    #             recipe_words.add(word)

    # len(recipe_words)

    model_path = get_model_path('gpt2', 'fp16_recipe_105791_lr_5e-5_wd_1e-1')

    eval_output_dir = os.path.join(CACHE_DIR, "gpt2_105791_title_prompt_output_recipes")
    pathlib.Path(eval_output_dir).mkdir(exist_ok=True)

    # Generate N recipes using random word in title vocab as input to model
    # Recipes are saved to tmp/gpt2_finetuned_output_recipes
    N = 500
    used_titles = []

    for i in tqdm(range(N)):
        print(f"> Recipe {i} of {N}")
        keyword = random.choice(tuple(recipe_titles))
        output_file_name = os.path.join(eval_output_dir, f"gpt2_recipe_{i}.txt")
        generate_file('gpt2', model_path, keyword, output_path=output_file_name, temperature=0.8, length=400, prefix='<BOS>', stop_token='<EOS>')

    # Save used titles for later reference
    title_list_file = os.path.join(eval_output_dir, f"titles.txt")
    with open(title_list_file, 'w') as f:
        f.writelines("%s\n" % t for t in used_titles)

  0%|          | 0/500 [00:00<?, ?it/s]

gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1
> Recipe 0 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 19:11:57.367308: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 19:11:58 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 19:12:04 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Slow Cooker Turkey and Potatoes', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  0%|          | 1/500 [00:13<1:52:19, 13.51s/it]

> Recipe 1 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 19:12:10.852940: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 19:12:12 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 19:12:18 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Endive & Gorgonzola', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  0%|          | 2/500 [00:27<1:52:21, 13.54s/it]

...

100%|█████████▉| 498/500 [1:51:49<00:27, 13.76s/it]

> Recipe 498 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 21:03:47.269854: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 21:03:48 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 21:03:54 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt="Grandma's Ground Beef Casserole", repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100%|█████████▉| 499/500 [1:52:03<00:13, 13.75s/it]

> Recipe 499 of 500
["--temperature='0.8'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2021-05-11 21:04:00.997123: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/11/2021 21:04:02 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/11/2021 21:04:08 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/Capstone/tmp/gpt2_fp16_recipe_105791_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Sweet Potatoes with Cuban-Style Beef Picadillo ', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100%|██████████| 500/500 [1:52:17<00:00, 13.48s/it]
